In [1]:
import pandas as pd 
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Reshape, Conv2D, MaxPooling2D, BatchNormalization, Dropout
import numpy as np
import cv2 
import os
from sklearn.model_selection import train_test_split 

In [53]:
image_size = (208,208)
batch_size = 32

In [3]:
def сorrecting_data():
    x = []
    y = []
    boxes = []
    df = pd.read_csv('_annotations.csv') 
    df['xmin'] = df['xmin'].apply(lambda x: x // 2)
    df['ymin'] = df['ymin'].apply(lambda x: x // 2)
    df['xmax'] = df['xmax'].apply(lambda x: x // 2)
    df['ymax'] = df['ymax'].apply(lambda x: x // 2)
    df['width'] = df['width'].apply(lambda x: x // 2)
    df['height'] = df['height'].apply(lambda x: x // 2)
    h = df['height']
    w = df['width']
    for x1,y1,x2,y2 in zip(df['xmin'], list(df['ymin']), 
                       list(df['xmax']), list(df['ymax'])):
        arr = [x1, y1, x2, y2]
        boxes.append(arr)
    return df, boxes

In [5]:
def data_metrics(df,boxes):
    images = []
    coordinates = []
    
    for index, boxes in df.iterrows():
        image = cv2.imread(index)
        images.append(image)
        coordinates.append(boxes)
    return images, coordinates

In [7]:
# Масштабирование и преобразование извлеченных областей
def data(images, coordinates, image_size):
    scaled_regions = []
    labels = []
    for image, coords in zip(images, coordinates):
        x, y, w, h = coords
        car_region = image[y:y+h, x:x+w]
        scaled_region = cv2.resize(car_region, image_size)
        scaled_regions.append(scaled_region)
        labels.append(1)  # Предполагая, что все извлеченные области - это машины
    # Преобразование входных данных в массивы NumPy
    scaled_regions = np.array(scaled_regions)
    labels = np.array(labels)
    return scaled_regions, labels


In [8]:
%cd DataSet_cars_train/train
df_train, boxes_train = сorrecting_data()
df_train = df_train.set_index('filename')
df = df_train.astype(str)
image_train, image_coords = data_metrics(df_train, boxes_train)
%cd ../..
%cd DataSet_cars_valid/valid
df_valid, boxes_valid = сorrecting_data()
df_valid = df_valid.set_index('filename')
df = df_valid.astype(str)
image_valid, image_coords = data_metrics(df_valid, boxes_valid)
%cd ../..

C:\Users\Matvey\Проект\Project\DataSet_cars_train\train
C:\Users\Matvey\Проект\Project
C:\Users\Matvey\Проект\Project\DataSet_cars_valid\valid
C:\Users\Matvey\Проект\Project


In [36]:
region_train, car_train = data(image_train,boxes_train, image_size)
region_valid, car_valid = data(image_valid,boxes_valid, image_size)

In [30]:
model = keras.Sequential([
    Conv2D(128, (3, 3), activation='relu', input_shape=(208, 208, 3)),  # свёрточный слой
    #попробуй по 2-3 свёрточных слоя сделать
    MaxPooling2D((2, 2), strides=2),  # уменьшение карт признаков
    Dropout(0.2),  # отключение 20 процентов нейронов

    Conv2D(128, (3, 3), activation='relu'),  # свёрточный слой
    MaxPooling2D((2, 2), strides=2),  # уменьшение карт признаков
    Dropout(0.2),  # отключение 20 процентов нейронов

    Conv2D(256, (3, 3), activation='relu'),  # свёрточный слой
    MaxPooling2D((2, 2), strides=2),  # уменьшение карт признаков
    Dropout(0.2),  # отключение 20 процентов нейронов

    Conv2D(256, (3, 3), activation='relu'),  # свёрточный слой
    MaxPooling2D((2, 2), strides=2),  # уменьшеи карт признаков
    Dropout(0.2),  # отключение 20 процентов нейронов

    Conv2D(512, (3, 3), activation='relu'),  # свёрточный слой
    MaxPooling2D((2, 2), strides=2),  # уменьшение карт признаков
    Dropout(0.2),  # отключение 20 процентов нейронов

    Conv2D(512, (3, 3), activation='relu'),  # свёрточный слой
    MaxPooling2D((2, 2), strides=2),  # уменьшение карт признаков
    Dropout(0.2),  # отключение 20 процентов нейронов

    Flatten(),
    Dense(512, activation='relu'),  # обычные нейроны
    Dropout(0.2),  # отключение 20 процентов нейронов
    Dense(256, activation='relu'),  # обычные нейроны
    Dropout(0.2),  # отключение 20 процентов нейронов
    Dense(128, activation='relu'),  # обычные нейроны
    Dropout(0.2),  # отключение 20 процентов нейронов
    Dense(1, activation='sigmoid'),  # выходной слой
])


In [31]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [37]:
train_data, val_data, train_labels, val_labels = train_test_split(
    region_train, car_train, test_size=0.1, random_state=42)

In [52]:
train_data[1][1]

array([[160, 160, 160],
       [161, 161, 161],
       [164, 164, 164],
       [164, 164, 164],
       [164, 164, 164],
       [164, 164, 164],
       [164, 164, 164],
       [164, 164, 164],
       [166, 166, 166],
       [167, 167, 167],
       [169, 169, 169],
       [170, 170, 170],
       [169, 169, 169],
       [168, 168, 168],
       [166, 166, 166],
       [163, 163, 163],
       [160, 160, 160],
       [157, 157, 157],
       [153, 153, 153],
       [152, 152, 152],
       [156, 156, 156],
       [159, 159, 159],
       [162, 162, 162],
       [166, 166, 166],
       [169, 169, 169],
       [171, 171, 171],
       [172, 172, 172],
       [172, 172, 172],
       [172, 172, 172],
       [172, 172, 172],
       [171, 171, 171],
       [169, 169, 169],
       [166, 166, 166],
       [163, 163, 163],
       [158, 158, 158],
       [155, 155, 155],
       [151, 151, 151],
       [151, 151, 151],
       [153, 153, 153],
       [155, 155, 155],
       [158, 158, 158],
       [162, 162

In [38]:
history = model.fit(train_data, train_labels, epochs = 10, batch_size = batch_size, validation_data = (val_data, val_labels))

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.